In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import spacy
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import PhraseMatcher
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import numpy as np
import textwrap

URL = 'https://www.greencarreports.com/news/first-drives'
page = requests.get(URL)

soup = BeautifulSoup(page.content,'html.parser')

#nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nlp = spacy.load('en_core_web_lg')

# Allow print to format None values
def xstr(s):
    if s is None:
        return ''
    else:
        return str(s)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [63]:
# Get list of reviews from the homepage

links = soup.find_all(class_='img-wrap',
                    href=True)

for a in links:
    print('https://www.greencarreports.com' + a['href'])
   

https://www.greencarreports.com/news/1134484_first-drive-review-2022-hyundai-ioniq-5-plugs-into-new-wave-not-the-mainstream
https://www.greencarreports.com/news/1133945_first-drive-review-2022-volvo-xc60-recharge-plug-in-hybrid-delivers-more-range-and-power
https://www.greencarreports.com/news/1133898_first-drive-review-2022-volvo-c40-recharge-trades-function-for-style
https://www.greencarreports.com/news/1133831_first-drive-review-2021-ford-mustang-mach-e-gt-upgrades-performance-without-losing-range
https://www.greencarreports.com/news/1133777_first-drive-review-2022-lucid-air-delivers-a-new-leading-edge-for-evs
https://www.greencarreports.com/news/1133704_review-rivian-r1t-photos-price-review-specs-info
https://www.greencarreports.com/news/1133504_first-drive-review-2022-polestar-2-dual-motor-revisits-the-value-equation
https://www.greencarreports.com/news/1133505_first-drive-review-2022-polestar-2-single-motor-beats-tesla-on-build-quality-remixes-priorities
https://www.greencarrepor

In [64]:
# Do it for all product reviews with guesses for car model
rev_list = []
keys_ = []
car_makes_all = []
car_makes_known = []
car_models_all = []
car_name_words = []

print(f"{'Year':{10}} {'Make':{15}} {'Model':{15}}")

for a in links:
    page_temp = requests.get('https://www.greencarreports.com' + a['href'])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    
    res = soup_temp.find(class_='article-title')
    title = res.get_text()
    year = re.findall("20\d\d", title)
    if (len(year)==1):
        year = int(year[0])
        x = re.search(str(year), title)
        rest = title[(x.span()[1]+1):]
        if len(rest)==0:
            # Atypical title format, skip
            continue
    else:
        # Atypical title format, skip
        continue 
    
    tokens = nltk.word_tokenize(rest)
    car_make = tokens[0]
    x2 = re.search(car_make, title)
    
    sen = nlp(rest)
    named_ents = sen.ents

    # Leaving the car_make in for entity detection in case it 
    # contextually helps identify the car model
    # Next, remove car make from the entity to get car model
    car_model = None
    for entity in named_ents:
        if (entity.label_== 'ORG') or (entity.label_ == 'PRODUCT'):
            if (entity.text == car_make):
                if entity.label_ == 'ORG':
                    car_makes_known = car_makes_known + [entity.text] 
            elif (car_model==None):
                str1 = entity.text
                car_model = str1.replace(car_make,'')

    if car_model==None:
        VBZ_word = None
      
        for i in range(1,len(sen)):
            # Find first instance of Verb to match singular third person
            if (sen[i].tag_=='VBZ') and (VBZ_word==None):   
                VBZ_word = sen[i].text
               
        if VBZ_word==None:      # Spacy tagger couldn't find it, try NLTK
            tagged = nltk.pos_tag(tokens)
            for t in tagged:
                if (t[1]=='VBZ') and (VBZ_word==None):    
                    VBZ_word = t[0]
        
        if VBZ_word==None:
            continue
        else:
            inds1 = title.find(car_make)
            inds2 = title.find(VBZ_word)
            car_model = title[inds1+len(car_make)+1:inds2-1]
    
    car_make = car_make.strip()
    if not(car_make[0].isupper()):    # Make is not capitalized - likely wrong
        car_make = None
    else:
        car_makes_all = car_makes_all + [car_make]
        car_name_words = car_name_words + [car_make.lower()]
        
    car_model = car_model.strip()
    if not(car_model[0].isupper() or car_model[0].isdigit()):  
        car_model = None
    else:
        car_models_all = car_models_all + [car_model]
        car_name_words = car_name_words + car_model.lower().split()
                
    keys_.append([year, car_make, car_model])
    rev_list.append('https://www.greencarreports.com' + a['href'])

    print(f'{xstr(year):{10}} {xstr(car_make):{15}} {xstr(car_model):{15}}')
    
# Get unique entries
car_makes_all = set(car_makes_all)
car_makes_known = set(car_makes_known)
car_makes_unknown = car_makes_all - car_makes_known
car_models_all = set(car_models_all)
car_name_words = set(car_name_words)

print(car_makes_all)
print(car_makes_known)
print(car_makes_unknown)


Year       Make            Model          
2022       Volvo           XC60 Recharge  
2022       Volvo           C40 Recharge   
2021       Ford            Mustang Mach-E GT
2022       Lucid           EVs            
2022       Rivian          R1T            
2022       Polestar        2 dual-motor   
2022       Polestar        2 single motor 
2021       Hyundai         Santa Fe Hybrid
2022       Audi            E-Tron GT      
2022       Mercedes-Benz   EQS            
2021       BMW             330e           
2022       Hyundai         Tucson Hybrid  
2021       Karma           GS-6           
2022       Chevrolet       Bolt EUV       
2021       Volkswagen      ID.4           
2021       Kia             Sorento Hybrid 
2021       Toyota          Mirai          
2021       Ford            Mustang Mach-E electric SUV
2021       Volvo           XC40 Recharge electric SUV
2021       Hyundai         Elantra Hybrid 
2021       Honda           Accord Hybrid  
2021       Toyota          Si

In [65]:
# Correct individual error
keys_[3][2] = 'Air'

In [66]:
car_name_words = set.union(car_name_words,{'tesla','s','y'})    # add tesla models to excluded words
car_name_words

{'2',
 '330e',
 'accord',
 'audi',
 'bmw',
 'bolt',
 'c40',
 'chevrolet',
 'dual-motor',
 'e-tron',
 'elantra',
 'electric',
 'eqs',
 'euv',
 'evs',
 'fe',
 'ford',
 'gs-6',
 'gt',
 'honda',
 'hybrid',
 'hyundai',
 'id.4',
 'karma',
 'kia',
 'lucid',
 'mach-e',
 'mercedes-benz',
 'mirai',
 'motor',
 'mustang',
 'polestar',
 'r1t',
 'recharge',
 'rivian',
 's',
 'santa',
 'sienna',
 'single',
 'sorento',
 'suv',
 'tesla',
 'toyota',
 'tucson',
 'volkswagen',
 'volvo',
 'xc40',
 'xc60',
 'y'}

In [67]:
car_makes_unknown.discard(None)

car_makes_unknown2 = {x for x in car_makes_unknown 
                      if not((len(nlp(x).ents)>0) and 
                             (nlp(x).ents[0].label_=='ORG'))}

print("Suggested car makes to add to NER:")
print(car_makes_unknown2)

Suggested car makes to add to NER:
{'Lucid', 'Rivian', 'Polestar', 'Karma'}


Human should approve suggested additions as names of car manufacturing companies. Rather than adding named entities by training the large English model, which sounds slow and tricky, I chose to use rule-based entity recognition.

In [68]:
ruler = nlp.add_pipe("entity_ruler")
for cm in list(car_makes_unknown2):
    ruler.add_patterns([{"label": 'ORG', "pattern": cm}])

# Test new labels
doc = nlp("New car models from Rivian")
for ent in doc.ents:
    print(ent.text , '->', ent.label_)

Rivian -> ORG


Decided it doesn't make sense to add model names without the make as they are often too generic, e.g., '2' or 'Denali'

In [69]:
# Find and combine text from the body of the review
text_all = ''
text_dict = {}

for i in range(0,len(rev_list)):
    page_temp = requests.get(rev_list[i])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    res = soup_temp.find(class_='article-body').find_all('p')
    text = ''
    for b in res:
        if b.parent.name != 'div':       # Gets rid of nested figure captions 
            text = text + '\n' + b.get_text(separator=' ')
    
    text_all = text_all + '\n' + text
    text_dict[tuple(keys_[i])] = text

In [70]:
# Top nouns
doc = nlp(text_all)
doc = [i.text.lower() for i in doc 
       if i.text not in STOP_WORDS and 
       i.text not in string.punctuation and 
       i.tag_=="NN"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top10_NN=doc.value_counts()[:20].index.tolist() 
top10_NN

['drive',
 'range',
 'system',
 'car',
 'wheel',
 'power',
 'motor',
 'mode',
 'battery',
 'mph',
 'vehicle',
 'inch',
 'time',
 'way',
 'road',
 'pedal',
 'speed',
 'highway',
 'steering',
 'driving']

In [71]:
bigram = list(ngrams(text_all.lower().split(),2))
bigram = (b for b in bigram if b[0] not in STOP_WORDS and 
          b[1] not in STOP_WORDS and 
          b[0] not in car_name_words and 
          b[1] not in car_name_words and 
          b[0] not in string.punctuation and 
          b[1] not in string.punctuation)
bigram = pd.Series(bigram)
top_2g = [w for w in bigram.value_counts().index if bigram.value_counts()[w]>5] 
top_2g = (' '.join(w) for w in top_2g)

top_2g = list(top_2g)
top_2g = pd.Series(top_2g)
keep = np.ones(len(top_2g),dtype=bool)

for i in range(1,len(top_2g)):
    #top_2g[i] = re.sub(r'[^\w\s]', '', top_2g[i])   # To agressive, you want to keep some apostrophes and hyphens
    top_2g[i] = top_2g[i].replace(',','')
    doc = nlp(top_2g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_2g = list(top_2g[keep].values)

# Filtering out geographical locations removes much of the list, e.g., "Los Angeles"

top_2g


['0-60 mph',
 'gas engine',
 'regenerative braking',
 'battery pack',
 'apple carplay',
 'one-pedal driving',
 'trip computer',
 'performance edition',
 'car reports',
 'green car',
 'all-wheel drive',
 'doesn’t feel',
 'fuel cell',
 'android auto',
 'feel like',
 'mpg combined',
 'epa estimated',
 'mph time',
 'brake pedal',
 'rear seat',
 '60 mph',
 'steering wheel',
 'federal tax',
 'drive modes',
 'heat pump',
 'wireless smartphone',
 'mpg city']

In [72]:
trigram = list(ngrams(text_all.lower().split(),3))
trigram = (b for b in trigram if b[0] not in STOP_WORDS and 
           b[1] not in STOP_WORDS and 
           b[2] not in STOP_WORDS and
           b[0] not in car_name_words and 
           b[1] not in car_name_words and 
           b[2] not in car_name_words and
           b[0] not in string.punctuation and 
           b[1] not in string.punctuation and
           b[2] not in string.punctuation)

trigram = pd.Series(trigram)
top_3g = [w for w in trigram.value_counts().index if trigram.value_counts()[w]>3] 
#top20t = trigram.value_counts()[:20].index.tolist() 
top_3g = (' '.join(w) for w in top_3g)

top_3g = list(top_3g)
top_3g = pd.Series(top_3g)
keep = np.ones(len(top_3g),dtype=bool)

for i in range(1,len(top_3g)):
    top_3g[i] = top_3g[i].replace(',','')
    doc = nlp(top_3g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_3g = list(top_3g[keep].values)

top_3g

['green car reports',
 '0-60 mph time',
 'drive (eqs 580)',
 'wireless apple carplay',
 '12.3-inch digital gauge',
 'total system output']

In [73]:
fourgram = list(ngrams(text_all.lower().split(),4))
fourgram = (b for b in fourgram if b[0] not in STOP_WORDS and 
           b[1] not in STOP_WORDS and 
           b[2] not in STOP_WORDS and
           b[3] not in STOP_WORDS and
           b[0] not in car_name_words and 
           b[1] not in car_name_words and 
           b[2] not in car_name_words and
           b[3] not in car_name_words and        
           b[0] not in string.punctuation and 
           b[1] not in string.punctuation and
           b[2] not in string.punctuation and
           b[3] not in string.punctuation)

fourgram = pd.Series(fourgram)
top_4g = [w for w in fourgram.value_counts().index 
          if fourgram.value_counts()[w]>=3] 
top_4g = (' '.join(w) for w in top_4g)

top_4g = list(top_4g)
top_4g = pd.Series(top_4g)
keep = np.ones(len(top_4g),dtype=bool)

for i in range(1,len(top_4g)):
    top_4g[i] = top_4g[i].replace(',','')
    doc = nlp(top_4g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_4g = list(top_4g[keep].values)

top_4g

['$7,500 federal ev tax']

In [74]:
# Collate single, bi-, trigram and four-grams NN's

def not_in_list (x, list_against):
    for w in list_against:
        if w.find(x)!=-1:
            return False
    else:
        return True
    
top10_NN_filt = list(filter
                     (lambda x: not_in_list(x, top_2g+top_3g+top_4g), 
                      top10_NN))
top_2g_filt = list(filter
                   (lambda x: not_in_list(x, top_3g+top_4g), 
                    top_2g))
top_3g_filt = list(filter
                   (lambda x: not_in_list(x, top_4g), 
                    top_3g))

top_grams = top_2g_filt + top_3g_filt + top_4g
top_grams

['gas engine',
 'regenerative braking',
 'battery pack',
 'one-pedal driving',
 'trip computer',
 'performance edition',
 'all-wheel drive',
 'doesn’t feel',
 'fuel cell',
 'android auto',
 'feel like',
 'mpg combined',
 'epa estimated',
 'brake pedal',
 'rear seat',
 'steering wheel',
 'federal tax',
 'drive modes',
 'heat pump',
 'wireless smartphone',
 'mpg city',
 'green car reports',
 '0-60 mph time',
 'drive (eqs 580)',
 'wireless apple carplay',
 '12.3-inch digital gauge',
 'total system output',
 '$7,500 federal ev tax']

In [75]:
top10_NN_filt

['range', 'power', 'motor', 'vehicle', 'way', 'road', 'speed', 'highway']

In [76]:
# Top adjectives

doc = nlp(text_all)
doc = [i.text.lower() for i in doc 
       if i.text not in STOP_WORDS and 
       i.text not in string.punctuation and 
       i.tag_=="JJ"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top20_JJ=doc.value_counts()[:20].index.tolist() 
top20_JJ

['electric',
 'rear',
 'new',
 'standard',
 'good',
 'hybrid',
 'long',
 'single',
 'big',
 'dual',
 'regenerative',
 'high',
 'different',
 'low',
 'short',
 'easy',
 'wireless',
 'large',
 'available',
 'quick']

In [83]:
# Top comparative adjectives

doc = nlp(text_all)
doc = [i.text.lower() for i in doc 
       if i.text not in STOP_WORDS and 
       i.text not in string.punctuation and 
       i.tag_=="JJR"]
doc=pd.Series(doc)
top20_JJR=doc.value_counts()[:20].index.tolist() 
top20_JJR

['better',
 'larger',
 'lower',
 'smaller',
 'closer',
 'shorter',
 'taller',
 'longer',
 'cheaper',
 'slower',
 'lighter',
 'bigger',
 'quicker',
 'heavier',
 'firmer',
 'sharper',
 'softer',
 'stiffer',
 'higher',
 'smoother']

In [77]:
pentagram=list(ngrams(text_all.lower().split(),5))
context = [i for i in pentagram if i[2] == 'better']
context

[('does', 'a', 'better', 'job', 'of'),
 ('offer', 'much', 'better', 'lap', 'times'),
 ('which', 'are', 'better', 'than', 'those'),
 ('weight', 'far', 'better', 'than', 'i'),
 ('interior', 'looks', 'better', 'in', 'my'),
 ('model', 'has', 'better', 'ratings', 'of'),
 ('to', 'get', 'better', 'body', 'control'),
 ('glass', 'for', 'better', 'noise', 'and'),
 ('be', 'a', 'better', 'solution', 'for'),
 ('it’s', 'just', 'better', 'than', 'its'),
 ('full', 'stop.', 'better', 'on', 'power,'),
 ('on', 'power,', 'better', 'on', 'efficiency,'),
 ('efficiency,', 'and', 'better', 'even', 'on'),
 ('was', 'noticeably', 'better', 'in', 'the'),
 ('got', 'a', 'better', 'backseat', 'than'),
 ('make', 'a', 'better', 'showing.', 'volkswagen'),
 ('and', 'has', 'better', 'drivability', 'than'),
 ('car', 'is', 'better', 'than', 'the'),
 ('the', 'road', 'better', 'than', 'tesla’s'),
 ('being', 'a', 'better', 'car', 'and'),
 ('still', 'a', 'better', 'experience.', 'versus'),
 ('get', 'a', 'better', 'suspension',

In [78]:
# Top adverbs
doc = nlp(text_all)
doc = [i.text.lower() for i in doc 
       if i.text not in STOP_WORDS and 
       i.text not in string.punctuation and 
       i.tag_=="RB"]
doc=pd.Series(doc)
top10_RB=doc.value_counts()[:10].index.tolist() 
top10_RB

['far',
 'nearly',
 'instead',
 'actually',
 'fast',
 'slightly',
 'forward',
 'later',
 'long',
 'simply']

In [79]:
# Top comparative adverbs
doc = nlp(text_all)
doc = [i.text.lower() for i in doc 
       if i.text not in STOP_WORDS and 
       i.text not in string.punctuation and 
       i.tag_=="RBR"]

doc=pd.Series(doc)
top10_RBR=doc.value_counts()[:10].index.tolist() 
top10_RBR

['longer',
 'better',
 'sooner',
 'faster',
 'closer',
 'later',
 'earlier',
 'slower',
 'more',
 'quicker']

In [80]:
f = open('opinion-lexicon-English\\negative-words.txt','r')

neg_line = f.readline()
while neg_line.startswith(';'):   # Skip Header
    neg_line = f.readline()
neg_txt = f.read()
neg_txt = nltk.word_tokenize(neg_txt)
f.close()

f = open('opinion-lexicon-English\\positive-words.txt','r')

pos_line = f.readline()
while pos_line.startswith(';'):
    pos_line = f.readline()
pos_txt = f.read()
pos_txt = nltk.word_tokenize(pos_txt)
f.close()
pos_txt

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [119]:
summary = []

for i in sent_tokenize(text_all[:2000]):
    include = False
    type_i = []
    for j in word_tokenize(i):
        if j in neg_txt:
            include = True
            type_i = type_i + ['Negative sentiment']
        elif j in pos_txt:
            include = True
            type_i = type_i + ['Positive sentiment']
        if j in top10_NN_filt:
            include = True
            type_i = type_i + ['Top nouns']
        if j in top_grams:
            include = True
            type_i = type_i + ['Top consecutive ngrams']
        if j in top10_RBR:
            include = True
            type_i = type_i + ['Top comparative adverbs']
        if j in top20_JJR: 
            include = True
            type_i = type_i + ['Top comparative adjectives']
    if include:
        i = " ".join(i.split())
        summary = summary + [i.replace('\n ', ' ')]
        type_i = set(type_i)            # Not currently used for anything

#my_wrap = textwrap.TextWrapper(width = 80, replace_whitespace=False)
#wrap_list = my_wrap.wrap(text=summary)
#for line in wrap_list:
#    print(line)

for sent in summary:
    print(textwrap.fill(sent, 80), '\n')

Bigger batteries for more electric range. 

While in Brussels, Belgium to drive the 2022 Volvo C40 Recharge I was given a
first look—and some first drive time—at the 2022 Volvo XC60 Recharge T8 plug-in
hybrid, which gets that plus more power, and an updated infotainment system. 

Volvo also anticipates a larger federal tax credit. 

The 2022 Volvo XC60 plug-in hybrid arrives in the U.S. later this year or early
in 2022. 

Power, range, battery, and one-pedal driving The largest change starts in the
center tunnel of the 2022 XC60 where the battery’s been upgraded to a “three-
layer pack,” as Volvo refers to it, with 18.7 kwh of capacity (of which 14.8 kwh
is usable) up from 11.6 kwh (of which 9.1 kwh was usable). 

A more powerful electric motor on the rear axle ups output from 87 hp to 142 hp
with total system output from the turbocharged and supercharged inline-4 and
PHEV system jumping from 400 hp and 472 lb-ft of torque to 455 hp and 523 lb-ft
of torque. 

While the gas engine isn’t

In [108]:
summary

'Bigger batteries for more electric range.\n\nWhile in Brussels, Belgium to  drive the 2022 Volvo C40 Recharge  I was given a first look—and some first drive time—at the 2022 Volvo XC60 Recharge T8 plug-in hybrid, which gets that plus more power, and an updated infotainment system.\n\nVolvo also anticipates a larger federal tax credit.\n\n'

In [109]:
i

'Volvo also anticipates a larger federal tax credit.'

In [86]:
print(textwrap.fill(summary, 80, replace_whitespace=False))

Bigger batteries for more electric range.

While in Brussels, Belgium to  drive
the 2022 Volvo C40 Recharge  I was given a first look—and some first drive
time—at the 2022 Volvo XC60 Recharge T8 plug-in hybrid, which gets that plus
more power, and an updated infotainment system.

Volvo also anticipates a larger
federal tax credit.

The 2022 Volvo XC60 plug-in hybrid arrives in the U.S.
later this year or early in 2022.

Power, range, battery, and one-pedal driving
The largest change starts in the center tunnel of the 2022 XC60 where the
battery’s been upgraded to a “three-layer pack,” as Volvo refers to it, with
18.7 kwh of capacity (of which 14.8 kwh is usable) up from 11.6 kwh (of which
9.1 kwh was usable).

A more powerful electric motor on the rear axle ups output
from 87 hp to 142 hp with total system output from the turbocharged and
supercharged inline-4 and PHEV system jumping from 400 hp and 472 lb-ft of
torque to 455 hp and 523 lb-ft of torque.

While the gas engine isn’t more

In [60]:
# Initialize
matcher = PhraseMatcher(sp.vocab)

word_list = top_2g_filt

patterns = [nlp(w) for w in word_list]

matcher.add("PATTERNS", patterns)
text_all = text_all.replace("\n", " ")
doc = nlp(text_all)
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start-5:end+5]
    print('...', span.text, '...','\n')


... Volvo also anticipates a larger federal tax credit. The 2022 Volvo ... 

... range, battery, and one-pedal driving The largest change starts in ... 

... of torque. While the gas engine isn’t more powerful, ... 

... for 2022. The larger battery pack and all-electric range ... 

... increase the XC60’s eligible federal tax credit status from the current ... 

... a fully electric vehicle: one-pedal driving capability thanks to regenerative braking ... 

... pedal driving capability thanks to regenerative braking. For 2022 the XC60 ... 

... about the upgraded PHEV: one-pedal driving has indeed arrived in Volvo ... 

... powerful electric motor and larger battery pack, if the accelerator pedal ... 

... Pure drive mode, the gas engine will kick in to provide ... 

... -pedal feature with more regenerative braking works, but the deceleration ... 

... the speed the stronger the regenerative braking. It wasn’t possible ... 

... at low speeds when the gas engine is running, and perhap